# Data Extraction

I download and loop through the *Annual Notifiable Disease Surveillance Reports* from [NS population health](https://novascotia.ca/dhw/populationhealth/). Each pdf contains a number of tables. For a starter I am after the tables in the appendix which includes all the data. Firstly the *Notifiable diseases reported in Nova Scotia in 2018 by Health Management Zone: Number of reports and crude rates per 100,000 population* table. This table breaks down case numbers and rates by health management zone in NS.

![NS Health Zones](../docs/NS_Health_Zones.png)

Following this I will look to extract the breakdown by age.

I need to be aware of document changes throughout the years to ensure that I am selecting the same tables or data.

In [1]:
import requests as r
import camelot as c
import pandas as pd
import os
from time import sleep
"Done with package loading"

'Done with package loading'

## Test with 2018

First I will test extraction from 2018 data located [here](https://novascotia.ca/dhw/populationhealth/documents/Annual-Notifiable-Disease-Surveillance-Report-2018.pdf). Then I will compare the files over the years to check for consistency and generalise my code for extraction.

The table that I want is located on page 27.

We extracted a table from the pdf. Let's take a look at it. Camelot extracts the table and automatically stores it as a pandas dataframe which is nice!

We've got a nice output! This is great. I had originally set `c.read_pdf` to it's default `flavor` of `"lattice"`, this resulted in a mess with a lot of newline characters and a very challenging output. Changing it to `"stream"` solved that problem and has generated a nice output.

There is now a lot of cleaning to do, but hopefully the other tables will be extracted in a similar way.

## Cleaning Aims

Let's discuss what I want for the output of each table.

* Columns
	* Class - e.g. `"Bloodborne Pathogens"`
	* Condition - e.g. `"Hepatitis C"`
	* Zone - e.g. `"Western"`
	* Year
	* Rate
	* n
* I want total for each class too (rates & counts)

These are the disease classes. Note that `"Direct Contact, Respiratory Routes,"` and `"and Through the Provision of Health Care"` is one class and is read in as two rows as the pdf has the value printed over two rows - `camelot` thinks it is two rows.

![2nd class name is over two rows in pdf](../docs/class_2_rows.png)

In [2]:
def fix_rows(dataf):
	
	while dataf[0][0] == "":
		dataf.drop([0], inplace=True)
		dataf.reset_index(drop=True, inplace=True)

	dataf[17][1] = "Total"
	for i in range(1,len(dataf.columns)):
		if dataf[i][2] == '':
			dataf[i][2] = dataf[i-1][2]
		if dataf[i][1] == '':
			dataf[i][1] = dataf[i-1][1]

	rowtest = dataf[2] == ''

	to_drop = []
	cl = [dataf[0][0]]
	for i in range(len(rowtest)-1):
		if rowtest[i] and rowtest[i+1]:
			to_drop.append(i)
			dataf[0][i+1] = dataf[0][i] + " " + dataf[0][i+1]
			cl.append(dataf[0][i+1])
		elif not(rowtest[i+1]):
			cl.append(cl[i])
		else:
			cl.append(dataf[0][i+1])
	
	dataf.insert(0,'class', cl)
	dataf.drop(to_drop, inplace=True)
	dataf.reset_index(drop=True, inplace=True)

	return dataf

In [3]:
def fix_columns(daf):
	coltest = (daf.iloc[4] == '').to_list()
	col_drop = []
	for i in range(len(coltest)):
		if coltest[i]:
			col_drop.append(i)

	col_drop = daf.columns[col_drop].to_list()
	daf.drop(col_drop, axis=1,inplace=True)
	
	colnames = ['class']
	for d in daf.columns.to_list()[1:]:
		# if d=='class':
		# 	continue
		st = daf[d][0] + " " + daf[d][1] + " " + daf[d][2]
		st = st.lower().strip().replace(" ","_")

		colnames.append(st.lower())
	daf.columns = colnames
	
	x = daf['class'] == "Condition"
	daf.drop(x*x.index,inplace=True)
	daf.head(5)
	for d in daf.columns.to_list()[2:]:
		daf[d] = pd.to_numeric(daf[d])

	daf.reset_index(drop=True, inplace=True)

	return daf[daf['class']!=daf['condition']]

In [4]:
def add_totals(df2):
	x = df2.groupby('class').sum()
	x['class'] = x.index
	x['condition'] = 'total'
	df2 = pd.concat([df2,x])
	return df2

In [7]:
def get_table_from_pdf(year, page):
	url = f"https://novascotia.ca/dhw/populationhealth/documents/Annual-Notifiable-Disease-Surveillance-Report-{year}.pdf"
	dl = r.get(url)
	file = "tmp.pdf"
	open(file, 'wb').write(dl.content)		# Cannot just push the content into camelot so create a temporary file
	tables = c.read_pdf(file, pages=f"{page}", flavor="stream")
	os.remove(file)
	f"Number of tables extracted: {tables.n}"
	dfrm = tables[0].df

	dfrm = fix_rows(dfrm)
	dfrm = fix_columns(dfrm)

	dfrm = dfrm[dfrm['condition']!='TOTAL']

	dfrm = add_totals(dfrm)
	dfrm.insert(0,'year',year)

	return dfrm

In [8]:
years = [2019, 2018, 2017, 2016, 2015]
pages = [13, 27, 28, 29, 29]
df = pd.DataFrame()

for y, p in zip(years, pages):
	sleep(10)
	df = pd.concat([df, get_table_from_pdf(y,p)])
	print(f"Got table for {y} from page {p}")

df.reset_index(drop=True, inplace=True)
df.head(6)

Got table for 2019 from page 13
Got table for 2018 from page 27
Got table for 2017 from page 28
Got table for 2016 from page 29
Got table for 2015 from page 29


,year,class,condition,western_n,western_rate,northern_n,northern_rate,eastern_n,eastern_rate,central_n,central_rate,total_n,total_rate
0,2019,Bloodborne Pathogens,Acquired Immune Deficiency Syndrome (AIDS),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019,Bloodborne Pathogens,Hepatitis B - Acute,0.0,0.0,1.0,0.7,0.0,0.0,3.0,0.6,4.0,0.4
2,2019,Bloodborne Pathogens,Hepatitis B - Chronic,1.0,0.5,1.0,0.7,5.0,3.1,15.0,3.2,22.0,2.3
3,2019,Bloodborne Pathogens,Hepatitis C,38.0,19.1,75.0,50.5,100.0,61.7,137.0,29.7,350.0,36.0
4,2019,Bloodborne Pathogens,Human Immunodeficiency Virus (HIV),0.0,0.0,1.0,0.7,4.0,2.5,14.0,3.0,19.0,2.0
5,2019,"Direct Contact, Respiratory Routes, and Throug...",Clostridium difficile,165.0,82.8,129.0,86.9,244.0,150.5,369.0,79.9,907.0,93.4


In [ ]:
df.to_csv("../data/zone_data.csv", index=False)